In [ ]:
import zipfile

# Extracting ZIP file for the images and CSV file
with zipfile.ZipFile('/content/Image_classification_data.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

#Imports

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from tensorflow.keras import datasets, layers, models
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization

#Exploratory Data Analysis

In [ ]:
data_main = pd.read_csv('/content/data_labels_mainData.csv')
data_main = data_main[['ImageName', 'cellType']]
data_main.head()

,ImageName,cellType
0,22405.png,0
1,22406.png,0
2,22407.png,0
3,22408.png,0
4,22409.png,0


In [ ]:
data_main['cellType'].value_counts()

2    4079
1    2543
0    1888
3    1386
Name: cellType, dtype: int64

In [ ]:
data_main['cellType'] = data_main['cellType'].astype('str')

Splitting the Data

In [ ]:
#Splitting data into train, test and validation
train_data, test_data = train_test_split(data_main, test_size=0.2, random_state=0)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=0)

print("Train data : {}, Val Data: {}, Test Data: {}".format(train_data.shape[0], val_data.shape[0], test_data.shape[0]))

Train data : 5937, Val Data: 1979, Test Data: 1980


#**Performing data augmentation on train data to balance the binary classes**

In [ ]:
data = pd.read_csv('/content/data_labels_mainData.csv')
data = data[['ImageName', 'cellType']]


In [ ]:
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42,stratify= data['cellType'])

In [ ]:
test_data

,ImageName,cellType
856,9162.png,1
9122,12135.png,2
359,181.png,1
8299,3653.png,2
2645,18326.png,0
...,...,...
7491,674.png,0
5832,19889.png,1
9131,12147.png,2
108,18872.png,1


In [ ]:
test_data['cellType'].value_counts()

2    408
1    254
0    189
3    139
Name: cellType, dtype: int64

In [ ]:
train_data['cellType'].value_counts()

2    3671
1    2289
0    1699
3    1247
Name: cellType, dtype: int64

In [ ]:
train_df = train_data

In [ ]:
train_df.head()

,ImageName,cellType
1702,13444.png,1
4611,16714.png,3
5790,19846.png,1
6982,1294.png,3
5423,11785.png,2


In [ ]:
import pandas as pd

# Assuming you have a DataFrame called 'df' with columns 'ImageName' and 'Class'

# Group the DataFrame by 'Class'
grouped_df = train_data.groupby('cellType')

# Create empty dictionaries to store the separated DataFrames
class_dfs = {}

# Iterate over the groups and create separate DataFrames for each class
for class_label, group in grouped_df:
    class_dfs[class_label] = group.drop('cellType', axis=1).copy()

# Access the separated DataFrames using the class label as the key
class0_df = class_dfs[0]
class1_df = class_dfs[1]
class2_df = class_dfs[2]
class3_df = class_dfs[3]

##Data augmentation

In [ ]:
# all the augmented images are stored in different floder so moving those folders into one folder 

import shutil

def move_augmented_files(source_dir, destination_dir):
    os.makedirs(destination_dir, exist_ok=True)

    for file_name in os.listdir(source_dir):
        if file_name.startswith("augmented"):
            source_path = os.path.join(source_dir, file_name)
            destination_path = os.path.join(destination_dir, file_name)
            shutil.move(source_path, destination_path)

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator

def augment_data(df, output_dir, num_augmentations=1):
    datagen = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

    augmented_image_paths = []

    for _, row in df.iterrows():
        image_name = row['ImageName']
        image_path = os.path.join(image_base_folder, image_name)

        # Load the image
        img = Image.open(image_path)
        img = img.resize((27, 27))

        # Get the file name without extension
        file_name = os.path.splitext(image_name)[0]

        # Perform data augmentation
        img_arr = np.array(img)
        img_arr = np.expand_dims(img_arr, axis=0)  # Add batch dimension for data generator

        datagen.fit(img_arr)

        augmented_image_name = f"augmented_{os.path.splitext(os.path.basename(image_path))[0]}.png"
        augmented_image_path = os.path.join(output_dir, augmented_image_name)

        for i, batch in enumerate(datagen.flow(img_arr, batch_size=1, save_to_dir=output_dir, save_prefix='', save_format='png')):
            augmented_image_temp_path = os.path.join(output_dir, f"temp_augmented_{i}.png")
            Image.fromarray(batch[0].astype(np.uint8)).save(augmented_image_temp_path)
            os.rename(augmented_image_temp_path, augmented_image_path)
            augmented_image_paths.append(augmented_image_path)
            break  # Generate only one augmented image per original image

            if i >= num_augmentations - 1:
                break  # Stop after generating the specified number of augmented images

    return augmented_image_paths

# Define your DataFrame 'df' with the 'ImageName' column
 
# Determine the number of augmented images to create based on the multiplier


## class 0 

In [ ]:
# Define the base path to the folder containing the images
image_base_folder = "/content/patch_images"

# Define the paths to the augmented dataset
augmented_dir = "/content/augmented_dataset10"
augmented_image_dir = os.path.join(augmented_dir)

# Create the augmented dataset directories if they don't exist
os.makedirs(augmented_image_dir, exist_ok=True)
# Data augmentation parameters


multiplier = 2  # Set the desired augmentation factor
num_augmentations = len(class0_df) * multiplier

# Perform data augmentation and get the augmented image names
augmented_image_paths = augment_data(class0_df, augmented_dir, num_augmentations)

# Update the 'ImageName' column with augmented image names
augmented_df = pd.DataFrame({'ImageName': [os.path.basename(image_path) for image_path in augmented_image_paths]})

# Concatenate the augmented DataFrame with the original majority class DataFrame

# Update the original DataFrame with the augmented image names and class labels
class0_df  = pd.concat([class0_df, augmented_df], ignore_index=True)

move_augmented_files(augmented_dir, image_base_folder)

In [ ]:
multiplier = 2  # Set the desired augmentation factor
num_augmentations = len(class0_df) * multiplier

# Perform data augmentation and get the augmented image names
augmented_image_paths = augment_data(class0_df, augmented_dir, num_augmentations)

# Update the 'ImageName' column with augmented image names
augmented_df = pd.DataFrame({'ImageName': [os.path.basename(image_path) for image_path in augmented_image_paths]})

# Concatenate the augmented DataFrame with the original majority class DataFrame

# Update the original DataFrame with the augmented image names and class labels
class0_df  = pd.concat([class0_df, augmented_df], ignore_index=True)

move_augmented_files(augmented_dir, image_base_folder)

## Class 1

In [ ]:
multiplier = 2  # Set the desired augmentation factor
num_augmentations = len(class1_df) * multiplier

# Perform data augmentation and get the augmented image names
augmented_image_paths = augment_data(class1_df, augmented_dir, num_augmentations)

# Update the 'ImageName' column with augmented image names
augmented_df = pd.DataFrame({'ImageName': [os.path.basename(image_path) for image_path in augmented_image_paths]})

# Concatenate the augmented DataFrame with the original majority class DataFrame

# Update the original DataFrame with the augmented image names and class labels
class1_1_df  = pd.concat([class1_df, augmented_df], ignore_index=True)

move_augmented_files(augmented_dir, image_base_folder)

In [ ]:
class1_df  = pd.concat([class1_df, class1_1_df], ignore_index=True)

## Class 2

In [ ]:
multiplier = 2  # Set the desired augmentation factor
num_augmentations = len(class2_df) * multiplier

# Perform data augmentation and get the augmented image names
augmented_image_paths = augment_data(class2_df, augmented_dir, num_augmentations)

# Update the 'ImageName' column with augmented image names
augmented_df = pd.DataFrame({'ImageName': [os.path.basename(image_path) for image_path in augmented_image_paths]})

# Concatenate the augmented DataFrame with the original majority class DataFrame

# Update the original DataFrame with the augmented image names and class labels
class2_df  = pd.concat([class2_df, augmented_df], ignore_index=True)

move_augmented_files(augmented_dir, image_base_folder)

## Class 3

In [ ]:
multiplier = 2  # Set the desired augmentation factor
num_augmentations = len(class3_df) * multiplier

# Perform data augmentation and get the augmented image names
augmented_image_paths = augment_data(class3_df, augmented_dir, num_augmentations)

# Update the 'ImageName' column with augmented image names
augmented_df = pd.DataFrame({'ImageName': [os.path.basename(image_path) for image_path in augmented_image_paths]})

# Concatenate the augmented DataFrame with the original majority class DataFrame

# Update the original DataFrame with the augmented image names and class labels
class3_3_df  = pd.concat([class3_df, augmented_df], ignore_index=True)

move_augmented_files(augmented_dir, image_base_folder)

In [ ]:
multiplier = 2  # Set the desired augmentation factor
num_augmentations = len(class3_3_df) * multiplier

# Perform data augmentation and get the augmented image names
augmented_image_paths = augment_data(class3_3_df, augmented_dir, num_augmentations)

# Update the 'ImageName' column with augmented image names
augmented_df = pd.DataFrame({'ImageName': [os.path.basename(image_path) for image_path in augmented_image_paths]})

# Concatenate the augmented DataFrame with the original majority class DataFrame

# Update the original DataFrame with the augmented image names and class labels
class3_3_3_df  = pd.concat([class3_3_df, augmented_df], ignore_index=True)

move_augmented_files(augmented_dir, image_base_folder)

In [ ]:
class3_df  = pd.concat([class3_3_3_df, class3_df], ignore_index=True)

In [ ]:
class0_df['cellType'] = '0'
class1_df['cellType'] = '1'
class2_df['cellType'] = '2'
class3_df['cellType'] = '3'

merged_df = pd.concat([class0_df, class1_df, class2_df, class3_df], ignore_index=True)


In [ ]:
merged_df['cellType'].value_counts()

2    7342
1    6867
0    6796
3    6235
Name: cellType, dtype: int64

In [ ]:
train_data, val_data = train_test_split(merged_df, test_size=0.10, random_state=0)
# Converting the data labels to string

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Real time data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')
val_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')

batch_size = 32

train_generator_2 = train_datagen.flow_from_dataframe(
        dataframe=train_data,
        directory='/content/patch_images',
        x_col="ImageName",
        y_col="cellType",
        target_size=(27, 27),
        batch_size=batch_size,
        class_mode='categorical')

valid_generator_2 = val_datagen.flow_from_dataframe(
        dataframe=val_data,
        directory='/content/patch_images',
        x_col="ImageName",
        y_col="cellType",
        target_size=(27, 27),
        batch_size=batch_size,
        class_mode='categorical')

# test data generator for evaluating the performance of a previously trained model on a test dataset
test_data['cellType'] = test_data['cellType'].astype('str')

test_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')

batch_size = 32

test_generator_2 = test_datagen.flow_from_dataframe(
        dataframe=test_data,
        directory='/content/patch_images',
        x_col="ImageName",
        y_col="cellType",
        target_size=(27, 27),
        batch_size=batch_size,
        class_mode='categorical')

Found 24516 validated image filenames belonging to 4 classes.
Found 2724 validated image filenames belonging to 4 classes.
Found 990 validated image filenames belonging to 4 classes.


# Using Extra Data Set to Improve Accuracy 

In [ ]:
import tensorflow as tf
from tensorflow import keras


best_model = keras.models.load_model('best_model.h5')

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Make predictions on the test generator
y_pred_prob = best_model.predict(test_generator_2)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Get true labels from the test generator
y_true = test_generator_2.classes

# Filter y_pred_prob based on correct predictions
correct_predictions = np.equal(y_pred, y_true)
y_pred_prob_correct = y_pred_prob[correct_predictions]

# Calculate the mean of predicted probabilities for correct predictions
y_pred_prob_mean = np.median(y_pred_prob_correct, axis=0)  # Mean along axis 0

# Generate classification report
class_names = list(test_generator_2.class_indices.keys())
report = classification_report(y_true, y_pred, target_names=class_names)

print(report)
print("Median of predicted probabilities for correct predictions:")
for class_idx, class_name in enumerate(class_names):
    print(f"{class_name}: {y_pred_prob_mean[class_idx]}")


31/31 [==============================] - 0s 12ms/step
              precision    recall  f1-score   support

           0       0.11      0.08      0.09       189
           1       0.28      0.35      0.31       254
           2       0.42      0.41      0.42       408
           3       0.13      0.12      0.12       139

    accuracy                           0.29       990
   macro avg       0.23      0.24      0.23       990
weighted avg       0.28      0.29      0.29       990

Median of predicted probabilities for correct predictions:
0: 0.040538571774959564
1: 0.0181454885751009
2: 0.7139264941215515
3: 0.0664956271648407


In [ ]:
data_extra = pd.read_csv('/content/data_labels_extraData.csv')
data_extra = data[['ImageName']]

In [ ]:
batch_size = 32

test_new_data_generator = test_datagen.flow_from_dataframe(
        dataframe=data_extra,
        directory='/content/patch_images',
        x_col="ImageName",
        y_col=None,
        target_size=(27, 27),
        batch_size=batch_size,
        class_mode=None)

Found 9896 validated image filenames.


In [ ]:
import numpy as np
import pandas as pd

# Make predictions on the validation generator
y_pred_prob = best_model.predict(test_generator_2)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Get true labels from the validation generator
y_true = test_generator_2.classes

# Filter y_pred_prob based on correct predictions
correct_predictions = np.equal(y_pred, y_true)
y_pred_prob_correct = y_pred_prob[correct_predictions]

# Calculate the median of predicted probabilities for correct predictions
class_medians = np.mean(y_pred_prob_correct, axis=0)

# For new unseen data, obtain predicted probabilities from the model
new_pred_prob = best_model.predict(test_new_data_generator)

# Calculate distances between predicted probabilities and median values
distances = np.abs(new_pred_prob - class_medians)

# Assign new unseen data to the class with the closest median value (smallest distance)
predicted_classes = np.argmin(distances, axis=1)

# Get the indices of the new data
new_data_indices = test_new_data_generator.index_array

# Create a DataFrame to store the data indices and their predicted classes
data_with_predictions = pd.DataFrame({'ImageName': data_extra['ImageName'], 'PredictedClass': np.nan})

# Populate the DataFrame with the data indices and predicted classes
data_with_predictions.loc[new_data_indices, 'PredictedClass'] = predicted_classes

# Print the DataFrame with predicted classes
print(data_with_predictions)


310/310 [==============================] - 3s 11ms/step
      ImageName  PredictedClass
0     22405.png               3
1     22406.png               1
2     22407.png               0
3     22408.png               0
4     22409.png               0
...         ...             ...
9891   1625.png               0
9892   1626.png               0
9893   1627.png               3
9894   1628.png               0
9895   1629.png               3

[9896 rows x 2 columns]


<ipython-input-107-4b20e0613704>:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_with_predictions.loc[new_data_indices, 'PredictedClass'] = predicted_classes


In [ ]:
data_with_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9896 entries, 0 to 9895
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ImageName       9896 non-null   object
 1   PredictedClass  9896 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 154.8+ KB


In [ ]:
data_with_predictions = data_with_predictions.rename(columns={"PredictedClass": "cellType"})


In [ ]:
data_with_predictions["cellType"] = data_with_predictions["cellType"].astype(str)

In [ ]:
train_data, val_data = train_test_split(data_with_predictions, test_size=0.10, random_state=0)
# Converting the data labels to string

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Real time data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')
val_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')

batch_size = 32

train_generator_new = train_datagen.flow_from_dataframe(
        dataframe=train_data,
        directory='/content/patch_images',
        x_col="ImageName",
        y_col="cellType",
        target_size=(27, 27),
        batch_size=batch_size,
        class_mode='categorical')

valid_generator_new = val_datagen.flow_from_dataframe(
        dataframe=val_data,
        directory='/content/patch_images',
        x_col="ImageName",
        y_col="cellType",
        target_size=(27, 27),
        batch_size=batch_size,
        class_mode='categorical')



Found 8906 validated image filenames belonging to 4 classes.
Found 990 validated image filenames belonging to 4 classes.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD

optimizer = SGD(learning_rate=learning_rate, momentum=0.9)

# Compile the model
best_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define the model checkpoint callback
checkpoint = ModelCheckpoint('best_model_new.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Train the model and save the best model based on validation accuracy
best_model.fit(train_generator_new, epochs=51, validation_data=valid_generator_new, callbacks=[checkpoint])


Epoch 1/51
276/279 [============================>.] - ETA: 0s - loss: 1.0476 - accuracy: 0.5095
Epoch 1: val_accuracy improved from -inf to 0.51212, saving model to best_model_new.h5
279/279 [==============================] - 7s 19ms/step - loss: 1.0484 - accuracy: 0.5083 - val_loss: 1.0228 - val_accuracy: 0.5121
Epoch 2/51
279/279 [==============================] - ETA: 0s - loss: 1.0181 - accuracy: 0.5173
Epoch 2: val_accuracy did not improve from 0.51212
279/279 [==============================] - 4s 16ms/step - loss: 1.0181 - accuracy: 0.5173 - val_loss: 1.0257 - val_accuracy: 0.5121
Epoch 3/51
277/279 [============================>.] - ETA: 0s - loss: 1.0180 - accuracy: 0.5167
Epoch 3: val_accuracy did not improve from 0.51212
279/279 [==============================] - 5s 19ms/step - loss: 1.0183 - accuracy: 0.5164 - val_loss: 1.0218 - val_accuracy: 0.5121
Epoch 4/51
278/279 [============================>.] - ETA: 0s - loss: 1.0157 - accuracy: 0.5166
Epoch 4: val_accuracy did not i

In [ ]:
from sklearn.metrics import classification_report
import numpy as np
from keras.utils import to_categorical

# Make predictions using the model on the generator data
# Make predictions using the model on the generator data
# Make predictions using the model on the generator data
y_pred_prob = best_model.predict(test_generator_2)
y_pred = np.argmax(y_pred_prob, axis=1)

# Concatenate true labels from the generator data
y_true = np.concatenate([test_generator_2[i][1] for i in range(len(test_generator_2))])
y_true = np.argmax(y_true, axis=1)

# Print the true class labels, predicted probabilities, and predicted class labels side by side
# Convert true labels to multilabel-indicator format if necessary

# Compute the classification report
report = classification_report(y_true, y_pred)
print(report)


31/31 [==============================] - 0s 10ms/step
              precision    recall  f1-score   support

           0       0.19      0.82      0.31       189
           1       0.28      0.07      0.12       254
           2       0.00      0.00      0.00       408
           3       0.13      0.11      0.12       139

    accuracy                           0.19       990
   macro avg       0.15      0.25      0.14       990
weighted avg       0.13      0.19      0.11       990



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
